In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

house = pd.read_csv('kc_house_data.csv')
house.columns

FileNotFoundError: [Errno 2] File kc_house_data.csv does not exist: 'kc_house_data.csv'

In [ ]:
house['sqft_living15']

In [ ]:
house.isnull().sum()

In [ ]:
house.insert(1,'recent_built',0)
house['recent_built'] = house.apply(lambda x : x['yr_built']
                                    if (x['yr_built']>=x['yr_renovated'])
                                    else x['yr_renovated'],axis=1)
house.drop(columns= ['yr_built','yr_renovated'])

In [ ]:
print("\nzipcode describe\n",house['zipcode'].describe())
print("\nlat describe\n",house['lat'].describe())
print("\nlong describe\n",house['long'].describe())

In [ ]:
corr = house[['price','date','sqft_lot','recent_built','bedrooms','grade','sqft_above','bathrooms','sqft_lot','condition','sqft_living','sqft_basement','floors','view','waterfront','sqft_living15','sqft_lot15']].corr()
corr['price']

select columns grade,sqft_above,bathrooms, sqft_living, view ,sqft_living15

In [ ]:
def find_outlier(data):
    Q1 , Q3 = np.percentile(data,[25,75])
    IQR = Q3 - Q1
    Over_outlier = Q3 + 1.5*IQR
    Low_outlier = Q1 - 1.5*IQR
    location = np.where((data>Over_outlier)|(data<Low_outlier))
    result = [list(location[0]),len(list(location[0]))]
    return result

In [ ]:
print("Grade column's Number of Outliers by IQR : " ,find_outlier(house['grade'])[1])
print("\nsqft_above column's Number of Outliers by IQR : " ,find_outlier(house['sqft_above'])[1])
print("\nbathrooms column's Number of Outliers by IQR : " ,find_outlier(house['bathrooms'])[1])
print("\nsqft_living column's Number of Outliers by IQR : " ,find_outlier(house['sqft_living'])[1])
print("\nview column's Number of Outliers by IQR : " ,find_outlier(house['view'])[1])
print("\nsqft_living15 column's Number of Outliers by IQR : " ,find_outlier(house['sqft_living15'])[1])

In [ ]:
plt.scatter(house['sqft_living'],house['price'])

In [ ]:
plt.scatter(house['sqft_above'],house['price'])

In [ ]:
plt.scatter(house['sqft_living15'],house['price'])

In [ ]:
house.boxplot(column=['price'], by='view')

In [ ]:
house.boxplot(column=['price'], by='grade')

In [ ]:
house.boxplot(column=['price'], by='bathrooms',figsize=(20,8))

In [ ]:
#number of outliers which is deleted
len(house[house['price']>3500000].index),len(house[house['sqft_living']>8000].index),len(house[house['sqft_above']>7000].index)

In [ ]:
house.drop(house[(house['price']>3000000) & (house['bathrooms']==1.75)|(house['price']>1000000)&(house['bathrooms']==0)].index,inplace=True)
house.drop(house[house['price']>3500000].index,inplace=True)
house.drop(house[house['sqft_living']>8000].index,inplace=True)
house.drop(house[house['sqft_above']>7000].index,inplace=True)



In [ ]:
plt.scatter(house['sqft_living'],house['price'])

In [ ]:
plt.scatter(house['sqft_above'],house['price'])

In [ ]:
plt.scatter(house['sqft_living15'],house['price'])

In [ ]:
house.boxplot(column=['price'], by='view')

In [ ]:
house.boxplot(column=['price'], by='grade')

In [ ]:
house.boxplot(column=['price'], by='bathrooms',figsize=(20,8))

In [ ]:

from patsy import dmatrices

y, X = dmatrices('price ~sqft_living+sqft_above+sqft_living15+view+grade+bathrooms',house, return_type = 'dataframe')
vif = pd.DataFrame()
vif["vif value"] = [variance_inflation_factor(X.values,i) for i in range(0,7)]
vif["explanatory variables"] = X.columns
vif

In [ ]:
house['intercept'] = 1

In [ ]:
import statsmodels.api as sm

X = house[['sqft_living15','sqft_above','view','grade','bathrooms']]
y = house['price']

X = sm.add_constant(X)
model = sm.OLS(y,X)
result = model.fit()
result.summary()

In [ ]:
# Error's Q-Q plot

import scipy.stats

residual = result.predict()
scipy.stats.probplot(residual, dist=scipy.stats.norm, plot=plt)
plt.show()

In [ ]:

house['sqft_above'].astype('float')
house['sqft_living15'].astype('float')
house['sqft_above'] = house['sqft_above']/100
house['sqft_living15'] = house['sqft_living15']/100


reg = sm.OLS(house['price'],house[['intercept','sqft_above','sqft_living15','view','grade','bathrooms']])
result = reg.fit()
result.summary()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X = house[['sqft_above','sqft_living15','view','grade','bathrooms']]
y = house['price']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
reg = LinearRegression()
reg.fit(X_train,y_train)
reg.score(X_test,y_test)
